In [8]:
!pip install nest_asyncio


error: externally-managed-environment

× This environment is externally managed
╰─> To install Python packages system-wide, try apt install
    python3-xyz, where xyz is the package you are trying to
    install.
    
    If you wish to install a non-Debian-packaged Python package,
    create a virtual environment using python3 -m venv path/to/venv.
    Then use path/to/venv/bin/python and path/to/venv/bin/pip. Make
    sure you have python3-full installed.
    
    If you wish to install a non-Debian packaged Python application,
    it may be easiest to use pipx install xyz, which will manage a
    virtual environment for you. Make sure you have pipx installed.
    
    See /usr/share/doc/python3.12/README.venv for more information.

note: If you believe this is a mistake, please contact your Python installation or OS distribution provider. You can override this, at the risk of breaking your Python installation or OS, by passing --break-system-packages.
hint: See PEP 668 for the detai

In [12]:
import simpy
import asyncio
import random

from abc import ABC, abstractmethod
from typing import Any, Dict, Callable, List

class Artifact(ABC):
    """A data object that agents interact with."""
    pass

class Context(ABC):
    """Environmental factors influencing actions."""
    data: Dict[str, Any]

    def __init__(self, **data):
        self.data = data

class Profile(ABC):
    """Behavioral profile encapsulating decision parameters for an Agent."""
    @abstractmethod
    def parameters(self) -> Dict[str, Any]:
        pass

class Policy(ABC):
    """Rule or strategy that conditions agent behavior."""
    @abstractmethod
    def evaluate(self, agent: 'Agent', artifact: Artifact, context: Context) -> bool:
        pass

class Event:
    """An occurrence triggered by an action."""
    def __init__(self, name: str, payload: Dict[str, Any] = None):
        self.name = name
        self.payload = payload or {}

# Event registry and decorator
_EVENT_HANDLERS: Dict[str, List[Callable[[Event], None]]] = {}

def on_event(event_name: str):
    """Decorator to subscribe a handler to an event."""
    def decorator(fn: Callable[[Event], None]):
        _EVENT_HANDLERS.setdefault(event_name, []).append(fn)
        return fn
    return decorator

def emit(event: Event):
    """Emit an event to all subscribers."""
    for handler in _EVENT_HANDLERS.get(event.name, []):
        handler(event)

from typing import Dict, Any, Callable
from abc import ABC

class Agent(ABC):
    def __init__(
        self,
        env: simpy.Environment,
        id: str,
        profile: Profile = None,
        policy: Policy = None,
        actions: Dict[str, Callable] = None
    ):
        self.env = env
        self.id = id
        self.profile = profile
        self.policy = policy
        self.actions = actions or {}
        self._register_actions()

    def _register_actions(self):
        for name in dir(self):
            if callable(getattr(self, name)) and not name.startswith("_"):
                self.actions.setdefault(name, getattr(self, name))

    async def perform(
        self,
        action: str,
        artifact: Artifact,
        context: Dict[str, Context],
        **kwargs
    ) -> Any:
        if action not in self.actions:
            raise ValueError(f"Action '{action}' not found for agent '{self.id}'")

        method = self.actions[action]

        result = await self._consume_async_generator(
            method(artifact, context, **kwargs)
        )

        ev = Event(f"{self.__class__.__name__}.{action}", {
            'agent_id': self.id,
            'artifact': artifact,
            'context': {k: v.dict() if hasattr(v, 'dict') else str(v) for k, v in context.items()},
            'result': result,
        })
        emit(ev)
        return result

    async def _consume_async_generator(self, async_gen) -> Any:
        """Consume the async generator and return the final value."""
        async for item in async_gen:
            # Return the last yielded value
            pass
        return item

# --- Example Implementations --- #

# Artifact subclass
class Product(Artifact):
    def __init__(self, sku: str, price: float):
        self.sku = sku
        self.price = price

class Inventory(Artifact):
    """Represents the inventory in a store."""
    
    def __init__(self):
        self.items = {}  # Mapping from product SKU to quantity

    def add_product(self, product: Product, quantity: int):
        """Add products to the inventory."""
        if product.sku in self.items:
            self.items[product.sku] += quantity
        else:
            self.items[product.sku] = quantity

    def remove_product(self, product: Product, quantity: int):
        """Remove products from inventory."""
        if product.sku in self.items and self.items[product.sku] >= quantity:
            self.items[product.sku] -= quantity
            return True
        return False

    def check_stock(self, product: Product) -> int:
        """Check the stock of a specific product."""
        return self.items.get(product.sku, 0)


# Context subclass
class MarketContext(Context):
    pass

class LeadTime(Context):
    """Represents the lead time for inventory restocking and product arrival."""
    
    def __init__(self, restocking_lead_time: int, delivery_lead_time: int):
        super().__init__(
            restocking_lead_time=restocking_lead_time, 
            delivery_lead_time=delivery_lead_time
        )
        self.restocking_lead_time = restocking_lead_time
        self.delivery_lead_time = delivery_lead_time

# Profile subclass
class CustomerProfile(Profile):
    def __init__(self, price_sensitivity: float):
        self.price_sensitivity = price_sensitivity
    def parameters(self) -> Dict[str, Any]:
        return {'price_sensitivity': self.price_sensitivity}

# Policy subclass
class PurchasePolicy(Policy):
    def __init__(self, threshold: float):
        self.threshold = threshold

    def evaluate(self, agent: Agent, artifact: Product, context: MarketContext) -> bool:
        sensitivity = agent.profile.parameters()['price_sensitivity']
        return artifact.price < self.threshold * sensitivity

# Agent subclass
class Customer(Agent):
    def __init__(self, env: simpy.Environment, id: str, profile: CustomerProfile, policy: PurchasePolicy):
        super().__init__(env, id, profile, policy)

    def define_actions(self) -> Dict[str, Callable]:
        return {
            'place_order': self.place_order,
            'add_to_cart': self.add_to_cart
        }

    async def place_order(self, store: "Store", sku: str, quantity: int, context: dict):
        return await store.process_order(self, sku, quantity, context)

    async def add_to_cart(self, product: Product, context: Dict[str, Context], quantity: int = 1):
        yield self.env.timeout(0.5)
        print(f"Customer {self.id} added {quantity}x {product.sku} to cart.")
        yield True

class InventoryManager(Agent):
    def __init__(
        self, env: simpy.Environment, id: str, 
        profile: Profile, policy: Policy, 
        inventory: Inventory
    ):
        super().__init__(env, id, profile, policy)
        self.inventory = inventory

    async def manage_inventory(
        self, 
        product: Product, context: Dict[str, Context], 
        quantity: int, action: str
    ) -> bool:
        await asyncio.sleep(0.1)
        if action == "add":
            self.inventory.add_product(product, quantity)
            return True
        elif action == "remove":
            self.inventory.remove_product(product, quantity)
            return True
        else:
            raise ValueError(f"Unknown action '{action}'")

class Salesman(Agent):
    def __init__(
        self, env: simpy.Environment, id: str, 
        profile: Profile, policy: Policy, 
        inventory: Inventory
    ):
        super().__init__(env, id, profile, policy)
        self.inventory = inventory

    async def sell(
        self, product: Product, context: MarketContext, quantity: int
    ) -> bool:
        await asyncio.sleep(1)
        if self.inventory.check_stock(product) >= quantity:
            self.inventory.remove_product(product, quantity)
            print(f"Salesman {self.id} sold {quantity}x {product.sku}.")
            return True
        print(f"Salesman {self.id} could not sell {quantity}x {product.sku} due to stock shortage.")
        return False

class Supplier(Agent):
    def __init__(
        self, env: simpy.Environment, 
        id: str, inventory: Inventory, 
        supply_time: float = 1.0
    ):
        super().__init__(env, id)
        self.inventory = inventory  # Supplier's own inventory
        self.supply_time = supply_time

    async def supply_product(self, product: Product, quantity: int, inventory: Inventory) -> bool:
        """Supply a specific store with a product."""
        print(f"Supplier {self.id} is supplying {quantity}x {product.sku} to {store.id}.")
        await asyncio.sleep(1)  # Simulate supply time
        
        # Check if supplier has enough product
        if self.inventory.check_stock(product) >= quantity:
            # Decrease the supplier's stock
            self.inventory.remove_product(product, quantity)

            # Supply time is simulated
            await asyncio.sleep(self.supply_time)

            # Restock the store's inventory
            inventory.add_product(product, quantity)
            print(f"Supplier {self.id} supplied {quantity}x{product.sku} to {store.id}.")
            return True
        else:
            print(f"Supplier {self.id} does not have enough {product.sku}.")
            return False

class Store(Agent):
    def __init__(
        self,
        env: simpy.Environment, id: str, 
        inventory: Inventory,
        suppliers: List[Supplier],
        num_salesmen: int = 5,
        restock_threshold: int = 10
    ):
        super().__init__(env, id)
        self.inventory = inventory
        self.suppliers = suppliers
        self.restock_threshold = restock_threshold

        # Staff members
        self.inventory_manager = InventoryManager(
            env, f"{id}_inv_mgr", 
            profile=None, policy=None, 
            inventory=inventory
        )
        self.salesmen = [
            Salesman(
                env, f"{id}_salesman_{i}", 
                profile=None, policy=None, 
                inventory=inventory
            )
            for i in range(num_salesmen)
        ]

    def process_order(self, customer, sku, quantity, context):
        def _inner():
            print(f"Store {self.id} received order from {customer.id} for {quantity}x{sku}")
            available_qty = self.inventory.check_stock(Product(sku, 0))
            if available_qty >= quantity:
                self.inventory.remove_product(Product(sku, 0), quantity)
                print(f"Store {self.id} fulfilled order for {customer.id}: {quantity}x{sku}")
                result = True
            else:
                print(f"Store {self.id} cannot fulfill order for {customer.id}: {quantity}x{sku}")
                result = False
            yield self.env.timeout(0)  # Optional delay
            return result
        return _inner()  # ✅ Return the generator, not the result


    async def restock_inventory(
        self, product: Product, context: Dict[str, Context], quantity: int
    ) -> bool:
        current_stock = self.inventory.check_stock(product)
        if current_stock >= self.restock_threshold:
            print(f"[{self.id}] No need to restock {product.sku} (stock: {current_stock}).")
            return False
        
        # Delegate to inventory manager who may coordinate with suppliers
        print(f"[{self.id}] Triggering restock for {product.sku}.")
        return await self.inventory_manager.manage_inventory(product, context, quantity, action="add")

    async def handle_sale(self, product: Product, context: Dict[str, Context], quantity: int) -> bool:
        # Random or round-robin strategy could be used here
        salesman = random.choice(self.salesmen)
        success = await salesman.sell(product, context, quantity)
        if success:
            print(f"[{self.id}] Sale successful: {quantity}x{product.sku}.")
        else:
            print(f"[{self.id}] Sale failed: Insufficient stock for {product.sku}.")
        return success

class RetailEcosystem:
    def __init__(self, env: simpy.Environment):
        self.env = env
        self.stores = []
        self.suppliers = []
        self.customers = []

    def add_store(self, store: Store):
        self.stores.append(store)

    def add_supplier(self, supplier: Supplier):
        self.suppliers.append(supplier)

    def add_customer(self, customer: Customer):
        self.customers.append(customer)

    def get_global_catalog(self):
        result = []
        for store in self.stores:
            for sku, quantity in store.inventory.items.items():
                if quantity > 0:
                    result.append({
                        "store_id": store.id,
                        "product": sku,
                        "available": True,
                    })
        return result

    def simulate(self, duration: int = 100):
        # Register all agent behaviors as processes
        for store in self.stores:
            self.env.process(self.store_behavior(store))

        for customer in self.customers:
            self.env.process(self.customer_behavior(customer))

        for supplier in self.suppliers:
            self.env.process(self.supplier_behavior(supplier))

        # Start simulation
        self.env.run(until=duration)

    def store_behavior(self, store: Store):
        while True:
            for product_sku, quantity in store.inventory.items.items():
                if quantity < store.restock_threshold:
                    product = Product(sku=product_sku, price=10.0)
                    context = {"time": {"lead_time": LeadTime(3, 2)}}
                    yield self.env.process(store.restock_inventory(product, context, quantity=50))
            yield self.env.timeout(10)  # Check every X time units

    def customer_behavior(self, customer: Customer):
        while True:
            catalog = self.get_global_catalog()
            if not catalog:
                print(f"No available products for {customer.id}")
                yield self.env.timeout(5)
                continue

            item = random.choice(catalog)
            store_id = item["store_id"]
            sku = item["product"]
            store = next((s for s in self.stores if s.id == store_id), None)
            if not store:
                yield self.env.timeout(1)
                continue

            quantity = random.randint(1, 5)
            context = {"market": {"lead_time": LeadTime(5, 2)}}

            # Now simulate customer placing order
            success = yield self.env.process(store.process_order(customer, sku, quantity, context))

            if success:
                print(f"Customer {customer.id} successfully ordered {quantity}x{sku} from {store_id}.")
            else:
                print(f"Customer {customer.id} failed to order {sku} from {store_id}.")

            yield self.env.timeout(random.randint(10, 20))  # Wait before next order


    def supplier_behavior(self, supplier: Supplier):
        while True:
            # Placeholder for supplier behavior (e.g., batching, responding to orders)
            yield self.env.timeout(1)

# Create simulation environment
env = simpy.Environment()

# Create products
product_1 = Product(sku="P123", price=5.0)
product_2 = Product(sku="P456", price=30.0)

# Create inventory, stores, suppliers, and customers
supplier_1_inventory = Inventory()
supplier_1_inventory.add_product(product_1, 100)
supplier_1 = Supplier(env, id="supplier_1", inventory=supplier_1_inventory)

# Create a product and add it to the supplier's inventory
supplier_2_inventory = Inventory()
supplier_2 = Supplier(env, id="supplier_2", inventory=supplier_2_inventory)

supplier_2_inventory.add_product(product_1, 50)
supplier_2_inventory.add_product(product_2, 200)

# Create a store
store_inventory = Inventory()
store_inventory.add_product(product_1, 20)

store = Store(
    env, id="store_x", 
    inventory=store_inventory, 
    suppliers=[supplier_1, supplier_2], 
    num_salesmen=3, 
    restock_threshold=10
)

# Create and add to Retail Ecosystem
duration = 30
ecosystem = RetailEcosystem(env)
ecosystem.add_store(store)

ecosystem.add_supplier(supplier_1)
ecosystem.add_supplier(supplier_2)

# Add a customer for testing
customer_1 = Customer(
    env=env, id="customer_1", 
    profile=CustomerProfile(price_sensitivity=0.8), 
    policy=PurchasePolicy(10.0)
)
customer_2 = Customer(
    env=env, id="customer_2", 
    profile=CustomerProfile(price_sensitivity=1), 
    policy=PurchasePolicy(30.0)
)

ecosystem.add_customer(customer_1)
ecosystem.add_customer(customer_2)

# Simulate the ecosystem
ecosystem.simulate(duration=duration)


Store store_x received order from customer_1 for 5xP123
Store store_x fulfilled order for customer_1: 5xP123
Store store_x received order from customer_2 for 4xP123
Store store_x fulfilled order for customer_2: 4xP123
Customer customer_1 successfully ordered 5xP123 from store_x.
Customer customer_2 successfully ordered 4xP123 from store_x.
Store store_x received order from customer_2 for 5xP123
Store store_x fulfilled order for customer_2: 5xP123
Customer customer_2 successfully ordered 5xP123 from store_x.
Store store_x received order from customer_1 for 2xP123
Store store_x fulfilled order for customer_1: 2xP123
Customer customer_1 successfully ordered 2xP123 from store_x.
[store_x] Triggering restock for P123.


AttributeError: 'coroutine' object has no attribute 'gi_frame'